In [29]:
import numpy as np
import numpy.linalg.linalg as LA
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from matplotlib.transforms import Affine2D
#from matplotlib.mlab import find
import matplotlib.patches as patches
from matplotlib.text import Annotation
import scipy.stats as stats
import statistics
import random
import pandas as pd
import math
import neat
import os
import pickle

# Simulation

In [34]:
twoPi = np.pi * 2

def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

class MultipleNRV(object):
    """Multiple independent normal random variables."""
    def __init__(self, size, loc=0., scale=1.):
        self.size = size
        self.mean, self.std = loc, scale
        self.twoVariance = 2 * self.std ** 2
    
    def pdf(self, xs):
        """Returns the probability density function value for a particular
        vector."""
        twoVar = self.twoVariance
        if twoVar == 0:
            return 1 if xs == self.mean else 0
        else:
            delta2 = (xs - self.mean) ** 2
            return np.product( np.exp( -delta2 / twoVar ) / np.sqrt( twoVar * np.pi) )        
            
    def sample(self):
        """Returns a vector sampled from the PDF."""
        loc, scale, n = self.mean, self.std, self.size
        return loc if scale == 0 else np.random.normal(loc, scale, size=self.size)                                                                        

class World(object):
    
    def __init__(self, sensor_angles=(0,), luminance=1.0, light_coords=(10.0, 0.0, -0.1), v_max=1.0, 
                 agent_radius=0.5, sensor_noise=0.01, motor_noise=0.5, dt=0.1, reachedLight=False, seed=None):
        self.sensors = np.array(sensor_angles)
        self.light_pos = np.array(light_coords)
        self.v_max = v_max
        self.agent_radius = agent_radius
        self.luminance = luminance
        self.dt = dt
        
        if seed is not None:
            np.random.seed(seed)

        # set up noise random variables
        sensor_sigma = sensor_noise * np.sqrt(dt)
        motor_sigma = motor_noise * np.sqrt(dt)
        self.sensor_rv = MultipleNRV(size=len(sensor_angles), scale = sensor_sigma)
        self.motor_rv = MultipleNRV(size=2, scale = motor_sigma)
                
    def sensor_pos(self, state):
        """Returns an array corresponding to a list of (x, y, 0) sensor 
        positions in world coordinates."""
        sensors, r = self.sensors, self.agent_radius
        x, y, theta = state
        n = len(sensors)
        
        result = np.zeros( (n, 3) )
        # copy robot x, y into sensors
        result[:,0:2] = state[0:2]
        
        angles = theta + sensors
        result[:,0] = r * np.cos(angles) + x
        result[:,1] = r * np.sin(angles) + y
        
        return result
        
    def sensor_input(self, state):
        """Returns an array of raw sensor input values for a particular
        agent state (position and orientation). These are calculated
        according to an inverse square distance law, and the agent's body
        can occlude a sensor reducing its input to zero.
        """        
        # various relevant parameters
        r, K = self.agent_radius, self.luminance
        # light position
        l_pos = self.light_pos        

        # unpack 3D position and heading from (x, y, theta) state
        pos, theta = np.array(tuple(state[0:2]) + (0,)), state[-1]        

        # positions in world coordinates of each sensor
        s_pos = self.sensor_pos(state)
        # array of distances of sensors from light source
        d_s = LA.norm(l_pos - s_pos, axis=1)
        
        # distance of light from robot's centre
        d_0 = LA.norm(l_pos - pos)

        # array of zeros or ones for each sensor according to whether the 
        # agent's body lies between the sensor and the light source
        not_occluded = (d_0**2 >= r**2 >= (d_s**2 - d_0**2))
        
        # light reaching each sensor
        return not_occluded * K / d_s **2                
        
    def sensor_transform(self, activation):
        """Returns a vector of sensor readings for a particular sensor input 
        value (activation) vector. Noise is usually applied to the activation 
        before applying the transform."""
        # rescale to (0, 1) interval, assuming activation is positive
        #return activation / (1 + activation)
        K, l_pos = self.luminance, self.light_pos
        # minimum distance is z coordinate of the light position
        d_min = l_pos[-1]
        
        # rescale activation to range between 0 and a_max
        # with midpoint around 
        a_max = K / ( d_min ** 2 )
        a = a_max / (1 + np.exp(5*(K/4 - activation)))
        
        #return 1 / (1 + np.exp(-activation))
        return activation
        #return np.sqrt(K / a)
    
    def sensor_inverse_transform(self, reading):
        """Returns the vector of sensor input values (activations) that would be 
        needed to produce the specified sensor reading. """
        return reading / (1 - reading)
    
    
    def sense(self, state):
        """Returns a vector of sensor reading values for a 
        particular agent state (position and orientation). 
        Noise is added to the raw luminance at the sensor's location
        and the result is rescaled to the (0, 1) interval.
        """
        activation = self.sensor_input(state) + self.sensor_rv.sample()
                        
        # and rescale to (0, 1) interval
        return self.sensor_transform(activation)
    
    def p_sensation(self, state, sensation):
        """Returns a probability density value for the likelihood of a 
        particular sensor reading vector given a particular agent state."""
        # invert rescaling operation to find the original activations 
        sensor_activation = self.sensor_inverse_transform(sensation)
        # determine the actual luminance at the sensors
        sensor_input = self.sensor_input(state)        
        
        # interrogate the RV object to get the PDF value
        return self.sensor_rv.pdf(sensor_input - sensor_activation)        

    def act(self, state, action):
        """Applies a motor activation vector to an agent state, and simulates 
        the consequences using Euler integration over a dt interval."""
        # noisily map the action values to a (-1, +1) interval
        motor_out = self.v_max * np.tanh(action) + self.motor_rv.sample()
        
        # calculate the linear speed and angular speed
        v = motor_out.mean()
        omega = (motor_out[1] - motor_out[0]) / (2.0 * self.agent_radius)
        
        # calculate time derivative of state
        theta = state[-1]
        deriv = [ v * np.cos(theta), v * np.sin(theta), omega ]
        
        # perform Euler integration
        return self.dt * np.array(deriv) + state
    
    
    
    def predictAct(self, state, action):
        
        #Map action values to (-1, 1), no noise
        motor_out = self.v_max * np.tanh(action)
        
        #Estimate the linear speed and angular speed
        v = motor_out.mean()
        omega = (motor_out[1] - motor_out[0]) / (2.0 * self.agent_radius)
        
        # calculate time derivative of state
        theta = state[-1]
        deriv = [ v * np.cos(theta), v * np.sin(theta), omega ]
        
        # perform Euler integration
        return self.dt * np.array(deriv) + state
        
        
        

    def random_state(self):
        """Returns a random initial state."""
        result = np.zeros(3)
        result[-1] = np.random.rand() * twoPi
        
        return result

    def distance_to_origin(self, poses):
        distance = math.sqrt((poses[-1][0]-poses[0][0])**2 + (poses[-1][1]-poses[0][1]) **2)
        return distance
    
    #Final pose distance from light
    def distance_to_light(self, poses):
        distance = LA.norm(self.light_pos[0:2] - poses[-1:,0:2], axis=1)
        return distance
    
    #Boolean if light reached or not
    def reached_light(self, poses):
        if self.distance_to_light(poses) > 1.5:
            return False
        else:
            return True
    
    def task1fitness(self, poses):
        if self.reached_light(poses):
            #print("Light Reached")
            return self.reached_light_at(poses) #removed -
        else:
            #print("Light not reached, distance: ", self.distance_to_light(poses))
            return -(self.distance_to_light(poses)**2) #Squared Error
        
        
    def task2fitness(self, poses):
        """Returns the fitness of the trajectory described by poses on 
        assignment task 1 (reaching the light source and returning to base)."""
        light_time = self.reached_light_at(poses)
        if light_time == np.inf:
            return -np.inf
        return -self.first_reached(poses, np.array([0, 0]), after=light_time) 
        
    def first_reached(self, poses, xy, after = 0, within = 1.5):
        after_index = int(np.floor(after / self.dt))
        ds = LA.norm(xy - poses[after_index:,0:2], axis=1)
        indices = np.nonzero(ds < within)[0]
        
        if len(indices) == 0:
            return np.inf
        else:
            fit = 5000-indices[0]
#             print("Fitness: {}".format(fit))
#             print("Type: {}".format(type(fit)))
            return int(fit)
        
    def reached_light_at(self, poses):
        return self.first_reached(poses, self.light_pos[0:2])
    
    
    def animate(self, poses, sensations, speedup=5):
        r, l_pos = self.agent_radius, self.light_pos
        x, y, theta = poses[0]
        
        # use an Ellipse to visually represent the agent's body
        body = patches.Ellipse(xy=(0, 0), width=2 * r, height=2 * r, fc='w', ec='k')
        # use a black dot to visually represent each sensor
        sensors = [ patches.Ellipse(xy=(r * np.cos(theta), r * np.sin(theta)), width=0.2, height=0.2, fc='b') for theta in self.sensors ]
        # use small rectangles to visually represent the motors
        motors = [ patches.Rectangle((-0.5*r, y), width = r, height = 0.2*r, color="black") for y in (-1.1*r, 0.9*r) ]
        # use a line to indicate the agent's orientation
        line = Line2D( (x, x + r * np.cos(theta)), (y, y + r * np.sin(theta)) )
        line = Line2D( (0, r), (0, 0) )
        # draw a line showing the agent's "trail" 
        trail = Line2D( [], [], color='r') 
        # display a clock
        clock = Annotation('', (0.8, 0.9), xycoords='axes fraction')    
        # use a yellow circle to visually represent the light
        light_r = patches.Ellipse(xy=l_pos[0:2], width=1, height=1, fc='y', ec='none')
        light = patches.Ellipse(xy=l_pos[0:2], width=0.25, height=0.25, fc='b')
        
        fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios': [10, 1] } )
        ax1.axis("equal")
        ax1.set_xlim([-15, 15])
        ax1.set_ylim([-15, 15])
        ax1.set_title("Click On Main Display To Pause / Unpause")
        
        ax2.set_title("Click On Sensor Graph To Change Time")
        
        tracker = ax2.axvline(0, 0, 1)
        paused = [ False ]
        last_index = [ -1 ]
        t_index = [ 0 ]
        
        if sensations is not None:
            times = np.arange(0, self.dt * len(sensations), self.dt)
            # plot the sensor values
            ax2.plot(times, sensations, 'k');
            # plot the ideal (noiseless) sensor values
            ideal = np.array([self.sensor_transform(self.sensor_input(pose)) for pose in poses[:-1]])
            ax2.plot(times, ideal, 'r')
    
        def draw(index):
            if not paused[0]:
                t_index[0] = t_index[0] + (index - last_index[0])
                t_index[0] = t_index[0] % len(poses)
    
            last_index[0] = index
                
            x, y, theta = poses[t_index[0]]
            tr = Affine2D().rotate(theta).translate(x, y) + ax1.transData
            
            agent_patches = (body, line) + tuple(sensors) + tuple(motors)
            
            for patch in agent_patches:
                patch.set_transform(tr);
                
            trail.set_data( poses[:t_index[0], 0], poses[:t_index[0], 1] )
            
            time = t_index[0] * self.dt
            tracker.set_xdata([time, time])
                
            clock.set_text("Time: %.02f" % time)
                    
            return (trail, light_r, light, clock, tracker) + agent_patches
        
        def init():
            result = draw(0)
            for artist in result:
                if artist is not tracker:
                    ax1.add_artist(artist)
            return result
    
        def onclick(event):
            if event.button == 1:
                # pause if the user clicks on the main figure
                if event.inaxes is ax1:
                    paused[0] = not paused[0]
                # edit time directly if the user clicks on the graph over time
                elif event.inaxes is ax2:
                    t_index[0] = (int) (event.xdata / self.dt)            
            
        def anim(index):
            return draw(index)
        
        
        ani = FuncAnimation(fig, anim, init_func=init, frames = None, interval=1000*self.dt/speedup, blit=True, save_count=len(poses))
        
        plt.show()
    
        fig.canvas.mpl_connect('button_press_event', onclick)
            
        return ani      
    
    
    #This is just used to visualise controllers evolved through NEAT
    def NEATsimulate(self, controller, interval=500.0):
        initialState = self.random_state()
        poses = [ initialState ] #Stores the actual x,y co-ordinates and bearings (inc. noise)
        estimatedPoses = [ initialState ] #stores an estimate of above as seen by controller (no noise)
        
        #State consists of [state pointer, bearings, maxIntensity, FirstPass, InitialBearing, StepsTaken, returnDistance]
        states = [[1, estimatedPoses, 0, True, 0, 0, 0]]
        
        sensations = [ ]
        actions = [ ]
        for i in range(int( interval / self.dt )):
            sensations.append(self.sense(poses[-1]))
            
            #Only used for NEAT approach
            if i == 0:
                action, state = controller(sensations[-1], [0,0], states[-1],self.dt)
            else:
                action, state = controller(sensations[-1],  actions[-1], states[-1], self.dt)
            
            actions.append(action)
            states.append(state)
            
            #Update position and estimated position
            poses.append(self.act(poses[-1], actions[-1]))
            estimatedPoses.append(self.predictAct(estimatedPoses[-1], actions[-1]))
            
            #Task 1 completion
            if self.reached_light(np.array(poses))==True:
                print("Light Reached!")
                self.reachedLight = True          
                
        return np.array(poses), np.array(sensations), np.array(actions), states
    
    
    
    def simulate(self, controller, interval=500.0):
        """Simulates the agent-environment system for the specified interval
        (in simulated time units) starting from a random state. Returns
        a (poses, sensations, actions, states) tuple where poses is a time array 
        of agent poses (position and orientation), sensations is a time array of 
        sensory readings, actions is a time array of motor activations, and
        states is a list of arbitrary internal controller state objects.
        
        Must be called with a controller function of the form 
        controller(sensation, state, dt) that returns a (action, state) tuple
        outputting motor activations and updated internal state in
        response to sensor readings.
        """
        initialState = self.random_state()
        poses = [ initialState ] #Stores the actual x,y co-ordinates and bearings (inc. noise)
        estimatedPoses = [ initialState ] #stores an estimate of above as seen by controller (no noise)
        
        
        #State consists of [state pointer, bearings, maxIntensity, FirstPass, InitialBearing, StepsTaken, returnDistance]
        states = [[1, estimatedPoses, 0, True, 0, 0, 0]]
        
        sensations = [ ]
        actions = [ ]
        for i in range(int( interval / self.dt )):
            sensations.append(self.sense(poses[-1]))
            action, state = controller(sensations[-1], states[-1], self.dt)
            actions.append(action)
            states.append(state)
            
            #Update position and estimated position
            poses.append(self.act(poses[-1], actions[-1]))
            estimatedPoses.append(self.predictAct(estimatedPoses[-1], actions[-1]))
            
            #Task 1 completion
            if self.reached_light(np.array(poses))==True and states[-1][0] == 3:
                print("Light Reached!")
                self.reachedLight = True
                states[-1][0] = 4
                
            #Terminating Conditions    
            if states[-1][0] == 6:
                print("Step Limit Hit")
                break
            elif self.distance_to_origin(poses) <= 1.5 and states[-1][0] == 5:
                print("ARRIVED HOME!")
                break
                
                
        return np.array(estimatedPoses), np.array(poses), np.array(sensations), np.array(actions), states
    

## Controllers

In [35]:
def constantController(sensors, state, dt):
    #print(sensors)
    return ([1, 1], None)

#Microbial Tumbling inspired controller
def ldependantController(sensors, state, dt):
    gain = 10
    return([gain*sensors[0], 10-gain*sensors[0]], None)

#Dead-Reckoning controller
def spinController(sensors, info, dt):
    state = info[0]
    print("State: ",state)
    estimatedPoses = info[1]
    maxIntensity = info[2]
    firstPass = info[3]
    
    
    #Spinning to find max intensity
    if state == 1:
        return SpinSearch(sensors, info) 
    
    #Returning to where max intensity was found
    elif state == 2:
        return LocateMaxIntensity(sensors,info)
    
    #Moving towards max intensity
    elif state == 3:
        return MoveTowardsTheLight(sensors, info)
    
    #Spin until bearing points home
    elif state == 4:
        print("FIND HOME STAGE")
        return FindHome(sensors, info)
    
    #Move towards home
    elif state == 5:
        print("GOING HOME STAGE")
        return GoHome(sensors, info) 


## SpinController Methods

In [38]:
def SpinSearch(sensors, info):
    state = info[0]
    estimatedPoses= info[1]
    currentBearing = info[1][-1][2]
    maxIntensity = info[2]
    firstPass = info[3]
    initialBearing = info[4]
    
    #Logs initial bearing on first iteration (when starting a spin)
    if firstPass == True:
        initialBearing = currentBearing
        maxIntensity = 0
        print("Initial bearing: ", initialBearing)
        firstPass = False
    
    #Records max intensity
    if sensors[-1] > maxIntensity:
            maxIntensity = sensors[-1]
    
    #Terminating conditions, proceed to next stage (returning to bearing of max intensity)
    if currentBearing >= initialBearing + 2 * math.pi or currentBearing <= initialBearing - 2 * math.pi:
        print("1 rotation completed")
        print("Max intensity: ", maxIntensity)
        state = 2
    return ([-0.5, 0.5], [state, estimatedPoses, maxIntensity, firstPass, initialBearing, info[5], info[6]])



def LocateMaxIntensity(sensors, info):
    maxIntensity = info[2]
    
    #Check if intensity nears previously recorded max
    threshold = 0.9
    if sensors[-1] >= maxIntensity * threshold:
        print("Max intensity found: ", sensors[-1])
        return ([0,0], [3,info[1],info[2],info[3], info[4], info[5], info[6]]) #Update stage to 3
    else:
        return ([-0.5, 0.5], info)
    
    
def MoveTowardsTheLight(sensors, info):
    maxIntensity = info[2]
    stepsTaken = info[5]
    stepSize = 50
    
    #Move forward until stepSize hit
    if stepsTaken < stepSize:
        print("Moving towards the light")
        return ([1,1], [info[0],info[1],info[2],info[3],info[4], stepsTaken+1, info[6]])
    else:
        print("Step limit hit, return to spin")
        return ([0,0], [1,info[1],info[2],True, info[4], 0, info[6]])#Toggles 'FirstPass' to true, to initialize spin starting angle
    
    
def FindHome(sensors, info):
    #The below are estimated values
    x = abs(info[1][-1][0])
    y = abs(info[1][-1][1])
    theta = (info[1][-1][2] * (180/math.pi))%360 #Convert to degrees
    
    #Pythagoras
    returnDistance = math.sqrt(x**2 + y**2)
    #Bearings adjusted as 0 is where you'd expect 90 to be, and measured anticlockwise from there
    if y < 0:
        returnBearing = (90 + math.degrees(math.atan(x/y))) % 360
    else:
        returnBearing = (180+ (90- math.degrees(math.atan(x/y)))  ) %  360

    #Checks estimated current bearing is within threshold (%) of calculated home bearing
    threshold = 0.05
    if theta > returnBearing*(1-threshold) and theta < returnBearing*(1+threshold):
        print("@@@@@@@@@@@@@@@@@ Pointing home! @@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        return ([0,0], [5,info[1],info[2],info[3],info[4], info[5], returnDistance])
    else:
        print("spinning to find home")
        return([-0.5,0.5], info)
    
def GoHome(sensors, info):
    stepsTaken = info[5]
    #The below are estimated values
    x = abs(info[1][-1][0])
    y = abs(info[1][-1][1])
    
    returnDistance = info[6] * (50/2)
    
    if stepsTaken < returnDistance*2:
        print("Moving Home")
        return ([1,1], [info[0],info[1],info[2],info[3],info[4], stepsTaken+1, info[6]])#Increment steps taken by 1
    else:
        print("Step limit hit! Am I home?")
        return ([0,0], [6,info[1],info[2],True, info[4], 0, info[6]])#Sets state to 6 to break simulation loop

## Run Simulation

In [40]:
# Visualisation
plt.ion()
%matplotlib qt

w = World()
estimatedPoses, poses, sensations, actions, states = w.simulate(spinController)
#print("Fitness on task 1: %f" % w.task1fitness(poses)) #was abs()
#print("Reached Light: ", w.reached_light(poses))
#print("Fitness on task 2: %f" % w.task2fitness(poses))
ani = w.animate(poses, sensations)

data = pd.DataFrame(np.concatenate([poses[:-1]], axis=1), columns=["x", "y", "theta"])

State:  1
Initial bearing:  0.6981689629425083
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
State:  1
1 rotation completed
Max intensity:  0.015704549623476286
State:  2
State:  2
State:  2
State:  2
Max intensity found:  0.014805740187655831
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards th

State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving towards the light
State:  3
Moving tow

GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving H

State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAGE
Moving Home
State:  5
GOING HOME STAG

In [8]:
#Create DataFrame
data = pd.DataFrame(np.concatenate([poses[:-1], estimatedPoses[:-1]], axis=1), columns=["X", "Y", "Theta", "Estimated X", "Estimated Y", "Estimated Theta"])
display(data)

,X,Y,Theta,Estimated X,Estimated Y,Estimated Theta
0,0.000000,0.000000,5.960807,0.000000,0.000000,5.960807
1,-0.017690,0.005909,6.056540,0.000000,0.000000,6.053231
2,-0.018690,0.006140,6.154020,0.000000,0.000000,6.145654
3,-0.012384,0.005321,6.240670,0.000000,0.000000,6.238078
4,-0.014653,0.005417,6.308963,0.000000,0.000000,6.330501
...,...,...,...,...,...,...
453,1.792304,-0.410103,28.135837,2.358419,1.329987,28.234854
454,1.720200,-0.400052,28.101199,2.282319,1.332993,28.234854
455,1.633295,-0.384854,28.079061,2.206219,1.335999,28.234854
456,1.563756,-0.371099,28.105944,2.130119,1.339005,28.234854


In [9]:
#Plot
ax = plt.gca()
#X
data.plot(kind='line',y='Estimated X', color='red', figsize=(14,6), ax=ax)
data.plot(kind='line',y='X', color='red', figsize=(14,6),ax=ax, linestyle='dashed', title=("A Graph to show the discrepancy between actual and estimated poses over time"))
#Y
data.plot(kind='line',y='Estimated Y', color='blue', ax=ax)
data.plot(kind='line',y='Y', linestyle='dashed',color='blue', ax=ax)
#Theta
data.plot(kind='line',y='Estimated Theta', color='orange', ax=ax)
data.plot(kind='line',y='Theta', linestyle='dashed',color='orange', ax=ax)

plt.xlabel("Time Step")
plt.ylabel("Value")

plt.grid()
plt.show()

# NEAT Attempt

In [22]:
    def NEATsimulate(self, controller, interval=500.0):
        initialState = self.random_state()
        poses = [ initialState ] #Stores the actual x,y co-ordinates and bearings (inc. noise)
        estimatedPoses = [ initialState ] #stores an estimate of above as seen by controller (no noise)
        
        #State consists of [state pointer, bearings, maxIntensity, FirstPass, InitialBearing, StepsTaken, returnDistance]
        states = [[1, estimatedPoses, 0, True, 0, 0, 0]]
        
        sensations = [ ]
        actions = [ ]
        for i in range(int( interval / self.dt )):
            sensations.append(self.sense(poses[-1]))
            
            #Only used for NEAT approach
            if i == 0:
                action, state = controller(sensations[-1], [0,0], states[-1],self.dt)
            else:
                action, state = controller(sensations[-1],  actions[-1], states[-1], self.dt)
            
            actions.append(action)
            states.append(state)
            
            #Update position and estimated position
            poses.append(self.act(poses[-1], actions[-1]))
            estimatedPoses.append(self.predictAct(estimatedPoses[-1], actions[-1]))
            
            #Task 1 completion
            if self.reached_light(np.array(poses))==True:
                print("Light Reached!")
                self.reachedLight = True          
                
        return np.array(estimatedPoses), np.array(poses), np.array(sensations), np.array(actions), states

In [10]:
best_networks = []
def eval_genomes(genomes, config):
    nets = []
    worlds = []
    ge = []
    for genome_ide, genome in genomes:
        genome.fitness = -math.inf
        net = neat.nn.RecurrentNetwork.create(genome, config)
        nets.append(net)
        worlds.append(World())
        ge.append(genome)
    
    global_best = 0
    for x, world in enumerate(worlds):
        poses = [world.random_state()]
        states = [None]
        sensations = []
        actions = []
        fit = -math.inf
        
        for i in range(int(500/0.1)):
            sensations.append(world.sense(poses[-1]))
            #Since we're using previous motor outputs as inputs, we must have a special case on the 1st iteration
            if i == 0:
                #Take 2 outputs from the network, after inputting (Sensor, x, y, Theta, motorL, motorR, velocity)
                output_1, output_2 = nets[worlds.index(world)].activate([sensations[-1]*5,poses[-1][0], poses[-1][1], poses[-1][2] % (math.pi*2),0,0,0])
                action = [output_1*10,output_2*10]
            else:
                c_left = actions[-1][0]
                c_right = actions[-1][1]
                left_wheel, right_wheel = nets[worlds.index(world)].activate([sensations[-1]*5,poses[-1][0], poses[-1][1], poses[-1][2] % (math.pi*2), c_left,c_right,statistics.mean([c_left,c_right]) ])
                action = [left_wheel*10,right_wheel*10]
            actions.append(action)         
            poses.append(world.act(poses[-1], actions[-1]))
            
            
        #Fitness evaluation
        print("Worlds: ", len(worlds))
        fit = world.task1fitness(np.array(poses))
        #print("ge: ", ge[worlds.index(world)])
        print("Fit: ", fit)
        if fit < 0:
            print("If Satisfied")
            ge[worlds.index(world)].fitness = int(fit)
            nets.pop(worlds.index(world))
            ge.pop(worlds.index(world))
            worlds.pop(worlds.index(world))
        else: # Save best model and fitness
            ge[worlds.index(world)].fitness = abs(fit)
            if abs(fit) > global_best:
                global_best = abs(fit)
                pickle.dump(nets[worlds.index(world)],open("best.pickle_CTRNN_1","wb"))

In [11]:
# Configuration file for a CTRNN neural network
__file__ = 'ctrnn-config.txt'
def run(config_file):
    """
    runs the NEAT algorithm to train a neural network.
    :param config_file: location of config file
    :return: None
    """
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    # Run for up to {50} m generations.
    winner = p.run(eval_genomes, 5)

    # show final stats
    print('\nBest genome:\n{!s}'.format(winner))
    
    
    winner_net = neat.nn.RecurrentNetwork.create(winner, config)
    
    return winner_net

if __name__ == '__main__':
    # Determine path to configuration file. This path manipulation is
    # here so that the script will run successfully regardless of the
    # current working directory.
    local_dir = os.path.dirname(__file__)
    config_path = os.path.join(local_dir, 'config-CTRNN-3.txt')
    win = run(config_path)


 ****** Running generation 0 ****** 

Worlds:  100
Fit:  [-129.41534187]
If Satisfied
Worlds:  99
Fit:  [-12786.41701961]
If Satisfied
Worlds:  98
Fit:  [-101.25989141]
If Satisfied
Worlds:  97
Fit:  [-96984.71069731]
If Satisfied
Worlds:  96
Fit:  [-158.74988742]
If Satisfied
Worlds:  95
Fit:  [-161198.19009204]
If Satisfied
Worlds:  94
Fit:  [-89.39532061]
If Satisfied
Worlds:  93
Fit:  [-110.38184607]
If Satisfied
Worlds:  92
Fit:  [-89.06747333]
If Satisfied
Worlds:  91
Fit:  [-113.08426549]
If Satisfied
Worlds:  90
Fit:  [-90.9980613]
If Satisfied
Worlds:  89
Fit:  [-2464.47361504]
If Satisfied
Worlds:  88
Fit:  [-4372.76664938]
If Satisfied
Worlds:  87
Fit:  [-144.03849442]
If Satisfied
Worlds:  86
Fit:  [-108.09245353]
If Satisfied
Worlds:  85
Fit:  [-189382.21921546]
If Satisfied
Worlds:  84
Fit:  [-4758.58919884]
If Satisfied
Worlds:  83
Fit:  [-291.56976]
If Satisfied
Worlds:  82
Fit:  [-88.13380888]
If Satisfied
Worlds:  81
Fit:  [-5802.88383337]
If Satisfied
Worlds:  80
Fi

Worlds:  102
Fit:  [-69887.85432946]
If Satisfied
Worlds:  101
Fit:  [-152.61809332]
If Satisfied
Worlds:  100
Fit:  [-86.85069528]
If Satisfied
Worlds:  99
Fit:  [-155357.97356432]
If Satisfied
Worlds:  98
Fit:  [-219048.42009698]
If Satisfied
Worlds:  97
Fit:  [-69.64488678]
If Satisfied
Worlds:  96
Fit:  [-62.69627756]
If Satisfied
Worlds:  95
Fit:  [-161184.10222298]
If Satisfied
Worlds:  94
Fit:  [-216510.21945603]
If Satisfied
Worlds:  93
Fit:  [-31329.73941502]
If Satisfied
Worlds:  92
Fit:  [-10356.85045544]
If Satisfied
Worlds:  91
Fit:  [-146293.27567672]
If Satisfied
Worlds:  90
Fit:  [-62.05581989]
If Satisfied
Worlds:  89
Fit:  [-22490.81079547]
If Satisfied
Worlds:  88
Fit:  [-1774.81421083]
If Satisfied
Worlds:  87
Fit:  [-430.69943748]
If Satisfied
Worlds:  86
Fit:  [-319.66651764]
If Satisfied
Worlds:  85
Fit:  [-142.48467572]
If Satisfied
Worlds:  84
Fit:  [-201511.15312402]
If Satisfied
Worlds:  83
Fit:  [-150.42524787]
If Satisfied
Worlds:  82
Fit:  [-70.4579129]
If

Worlds:  91
Fit:  [-84.01360329]
If Satisfied
Worlds:  90
Fit:  [-178231.6360197]
If Satisfied
Worlds:  89
Fit:  [-257.13668314]
If Satisfied
Worlds:  88
Fit:  [-182913.05704978]
If Satisfied
Worlds:  87
Fit:  [-233147.9441088]
If Satisfied
Worlds:  86
Fit:  [-150962.47837914]
If Satisfied
Worlds:  85
Fit:  [-403.9981409]
If Satisfied
Worlds:  84
Fit:  [-60750.11440079]
If Satisfied
Worlds:  83
Fit:  [-169.8225087]
If Satisfied
Worlds:  82
Fit:  [-69287.66194485]
If Satisfied
Worlds:  81
Fit:  [-116228.40350647]
If Satisfied
Worlds:  80
Fit:  [-61573.66580956]
If Satisfied
Worlds:  79
Fit:  [-25618.87249181]
If Satisfied
Worlds:  78
Fit:  [-48618.97354191]
If Satisfied
Worlds:  77
Fit:  [-6345.31305247]
If Satisfied
Worlds:  76
Fit:  [-8830.94644899]
If Satisfied
Worlds:  75
Fit:  [-1563.96531924]
If Satisfied
Worlds:  74
Fit:  [-172.40066781]
If Satisfied
Worlds:  73
Fit:  [-181.26891079]
If Satisfied
Worlds:  72
Fit:  [-80.95667343]
If Satisfied
Worlds:  71
Fit:  [-105.51973532]
If S

## Visualise Best Model

In [45]:
def controller(sensors, action, state, dt):
    c_left = action[0]
    c_right = action[1]
    n_left, n_right = win.activate([sensors*5,poses[-1][0], poses[-1][1], poses[-1][2] % (math.pi*2),c_left,c_right, statistics.mean([c_left,c_right])])
    action = [n_left*10,n_right*10]
    return (action, None)

In [47]:
plt.ion()
%matplotlib qt

w = World()
poses, sensations, actions, states = w.NEATsimulate(controller)
print("Fitness on task 1: %f" % w.task1fitness(poses))
print("Fitness on task 2: %f" % w.task2fitness(poses))
ani = w.animate(poses, sensations)

data = pd.DataFrame(np.concatenate([poses[:-1], sensations, actions], axis=1), columns=["x", "y", "theta", "sensor", "l_motor", "r_motor"])

Fitness on task 1: -132.971993
Fitness on task 2: -inf
